In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('Combined Data.csv')

In [13]:
df.head()

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety


In [15]:
#All the mental health feelings in the dataset. 
df.status.unique()

array(['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar',
       'Personality disorder'], dtype=object)

In [17]:
oh = OneHotEncoder(sparse_output=False)
encoder = oh.fit_transform(df[['status']])

encoded_df = pd.DataFrame(encoder, columns=oh.get_feature_names_out(['status']))

In [19]:
df['statement'] = df['statement'].str.replace('[^a-zA-Z]+', ' ', regex=True)

In [20]:
df['statement'] = df['statement'].apply(lambda x: str(x).lower())

In [23]:
vectorizer = TextVectorization(max_tokens=50000, output_mode='int', output_sequence_length=1800)

In [25]:
vectorizer.adapt(df['statement'])

In [26]:
X = vectorizer(df['statement'])

In [29]:
X

<tf.Tensor: shape=(53043, 1800), dtype=int64, numpy=
array([[ 585,    6, 4760, ...,    0,    0,    0],
       [ 920,  630,  943, ...,    0,    0,    0],
       [  33,  268,   96, ...,    0,    0,    0],
       ...,
       [  12,   68,   86, ...,    0,    0,    0],
       [1244, 1433,  967, ...,    0,    0,    0],
       [   2,   13,   63, ...,    0,    0,    0]])>

In [31]:
encoded_df

,status_Anxiety,status_Bipolar,status_Depression,status_Normal,status_Personality disorder,status_Stress,status_Suicidal
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
53038,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53039,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53040,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53041,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
y = np.array(encoded_df)

In [35]:
y = tf.convert_to_tensor(y)

In [37]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [39]:
dataset = dataset.batch(32)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
dataset = dataset.shuffle(len(dataset))
dataset = dataset.cache()

In [41]:
dataset_length = len(dataset)
train_size = int(dataset_length * 0.7)  # 70% for training
val_size = int(dataset_length * 0.2)  # 15% for validation
test_size = dataset_length - (train_size + val_size)  # 

train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size).take(val_size)
test_ds = dataset.skip(train_size+val_size).take(test_size)

In [43]:
model = Sequential()
model.add(Embedding(50000+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='sigmoid'))

model.compile(loss="BinaryCrossentropy", optimizer='Adam')

model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
1159/1160 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - loss: 0.3219

2024-12-27 15:00:53.183003: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1160/1160 ━━━━━━━━━━━━━━━━━━━━ 437s 376ms/step - loss: 0.3218 - val_loss: 0.2022
Epoch 2/10


2024-12-27 15:01:05.918580: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1159/1160 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step - loss: 0.2134

...

2024-12-27 15:16:46.863023: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1160/1160 ━━━━━━━━━━━━━━━━━━━━ 475s 409ms/step - loss: 0.1901 - val_loss: 0.1485
Epoch 4/10


2024-12-27 15:17:00.078204: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


 817/1160 ━━━━━━━━━━━━━━━━━━━━ 2:20 409ms/step - loss: 0.1749

KeyboardInterrupt: 

In [88]:
text = vectorizer('im restless and restless its been a month now boy what do you mean')

prediction = model.predict(np.expand_dims(text, 0))

np.round(prediction)

#This model predicts that this statement demonstrates anxiety, but not 
#necessarily bipolar disorder, depression, personality disorder, stress, suicidal intentions.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array([[1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [80]:
encoded_df

#This dataset shows what feelings that a certain piece of text might signify in terms of mental health conditions.

#For example the first entry says 1 for status_Anxiety. 
#This means that the corresponding statement shows anxiety but not any other mental health issue.

,status_Anxiety,status_Bipolar,status_Depression,status_Normal,status_Personality disorder,status_Stress,status_Suicidal
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
53038,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53039,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53040,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53041,1.0,0.0,0.0,0.0,0.0,0.0,0.0
